# Predictive Modeling
### Kwame V. Taylor

I will set the baseline and create the first ML model to predict song popularity.

## Set up Environment

In [1]:
import numpy as np
import pandas as pd

from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error, explained_variance_score, mean_absolute_error
from sklearn.linear_model import LinearRegression, TweedieRegressor
from sklearn.feature_selection import RFE
from sklearn.preprocessing import PolynomialFeatures

import warnings
warnings.filterwarnings("ignore")

In [2]:
from prepare import handle_nulls
from preprocessing import spotify_split, split_df, scale_data

## Acquire data

In [3]:
df = pd.read_csv('full-playlist.csv', index_col=0)

In [4]:
df.head()

,artist,album,release_date,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,explicit,popularity,disc_number
0,Tay-K,TRAPMAN,2020-07-12,TRAPMAN,6mecZbKK3JDeMdFRNxsCV5,0.792,0.594,2.0,-8.544,1.0,0.3000,0.000000,0.244,0.351,82.512,232803.0,4.0,True,43.0,1.0
1,Lil Wyte,Doubt Me Now,2003-03-04,Oxy Cotton,5PtMwNq8Dp31uYdGGacVJE,0.816,0.578,9.0,-6.912,1.0,0.2330,0.000000,0.114,0.265,148.077,193920.0,4.0,True,61.0,1.0
2,Kamelen,KINGPIN SLIM,2019-11-29,Kingpin O.G - Remix,6s8EhlBn2PIoESylkXnwYc,0.649,0.798,0.0,-6.450,0.0,0.1450,0.000000,0.409,0.717,160.011,254390.0,4.0,True,22.0,1.0
3,Waka Flocka Flame,Flockaveli,2010-10-01,Grove St. Party (feat. Kebo Gotti),2e9EZ2V5QGGZPMJacO3y0Y,0.705,0.702,0.0,-4.783,0.0,0.1080,0.000000,0.364,0.771,140.059,250493.0,4.0,True,62.0,1.0
4,Project Pat,Mista Don't Play: Everythangs Workin',2001-02-13,Don't Save Her (feat. Crunchy Black),3ZRd5Z0fiYtASLdEPPb16m,0.838,0.793,11.0,-5.470,0.0,0.0773,0.000001,0.106,0.800,160.003,261933.0,4.0,True,45.0,1.0


In [5]:
df.shape

(6074, 20)

## Prepare data

In [6]:
# handle null values
df = handle_nulls(df)

In [7]:
# check for nulls
df.isna().sum()

artist              0
album               0
release_date        0
track_name          0
track_id            0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
instrumentalness    0
liveness            0
valence             0
tempo               0
duration_ms         0
time_signature      0
explicit            0
popularity          0
disc_number         0
dtype: int64

In [8]:
# check data types
df.dtypes

artist               object
album                object
release_date         object
track_name           object
track_id             object
danceability        float64
energy              float64
key                 float64
loudness            float64
mode                float64
speechiness         float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
duration_ms         float64
time_signature      float64
explicit               bool
popularity          float64
disc_number         float64
dtype: object

Note to self: After MVP we need to convert release_data into a Timestamp.

## Preprocess data

In [9]:
# chose features for MVP modeling
df = df.drop(columns=['artist', 'album', 'release_date', 'track_name', 'track_id'])

In [10]:
# show features
df.columns

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms',
       'time_signature', 'explicit', 'popularity', 'disc_number'],
      dtype='object')

In [11]:
# split the data
X_train, y_train, X_validate, y_validate, X_test, y_test, train, validate, test = spotify_split(df, 'popularity')
train.head()

Shape of train: (4250, 14) | Shape of validate: (912, 14) | Shape of test: (911, 14)
Percent train: 70.0        | Percent validate: 15.0       | Percent test: 15.0


,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,explicit,popularity,disc_number
3072,0.646,0.595,10.0,-6.709,0.0,0.0512,0.000006,0.0527,0.772,73.973,238880.0,4.0,False,62.0,1.0
5674,0.839,0.335,9.0,-14.418,1.0,0.1750,0.000009,0.0967,0.566,127.053,151181.0,4.0,True,36.0,1.0
4005,0.517,0.903,10.0,-6.333,0.0,0.5680,0.000000,0.6900,0.643,84.792,196338.0,4.0,True,21.0,1.0
1130,0.923,0.604,11.0,-6.671,0.0,0.2450,0.000000,0.2720,0.826,145.062,139741.0,4.0,True,18.0,1.0
5441,0.595,0.407,8.0,-11.978,1.0,0.1930,0.000053,0.1620,0.452,144.938,126685.0,4.0,True,59.0,1.0


In [12]:
# scale the data
X_train_scaled, X_validate_scaled, X_test_scaled = scale_data(train, validate, test, 'popularity', 'MinMax')

## Set the baseline

In [13]:
#np.median(y_train)
np.mean(y_train)

38.46776470588235

In [14]:
#baseline = y_train.median()
baseline = y_train.mean()

baseline_rmse_train = round(mean_squared_error(y_train, np.full(len(y_train), baseline))**1/2, 6)
print('RMSE (Root Mean Square Error) of Baseline on train data:\n', baseline_rmse_train)
baseline_rmse_validate = round(mean_squared_error(y_validate, np.full(len(y_validate), baseline))**1/2, 6)
print('RMSE (Root Mean Square Error) of Baseline on validate data:\n', baseline_rmse_validate)

RMSE (Root Mean Square Error) of Baseline on train data:
 259.24048
RMSE (Root Mean Square Error) of Baseline on validate data:
 265.302576


Mean performed better than median.

Our baseline prediction of popularity will be ```38.49411764705882```, with an RMSE of ```259.343806``` on the train data and ```263.89558``` on the validate data.

## Model 1 - Ordinary Least Squares (OLS) using Linear Regression

In [15]:
#show available features
X_train_scaled.columns

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms',
       'time_signature', 'explicit', 'disc_number'],
      dtype='object')